# Week 3 Wrap-Up Exercises

**Note**: 

> This exercise has been written out in something called a Jupyter Notebook. We'll discuss Jupyter Notebooks in more detail later in this specialization—they are very a powerful tool for data science communication!—but for the time being, the notebook is just a convenient way for us to write out the exercise. You don't need to *do* anything with the notebook except read its contents—just use write your Python code in a regular `.py` file.

**WARNING:**

> When asked to round your answers to a certain number of decimals, do *not* round any results until you've finished your computations and have your final answer! For example, if you were to calculate the average hourly wage for workers, and you did so by first calculating the average weekly salary of workers and the average hours worked per week, then divided the first number by the second, you should NOT round the average weekly salary of workers or the average hours worked per week. Rounding intermediate results can lead to compounding errors that cause problems for the autograder.


In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.



## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Please load it.

In [33]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

# Download the data
acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/blob/master"
    "/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta?raw=True"
)

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [34]:
acs = acs[["age", "sex", "educ", "inctot"]].copy()

In [35]:
acs.sample(10)

,age,sex,educ,inctot
58833,19,male,grade 12,0
98376,22,female,1 year of college,0
261120,24,female,2 years of college,0
156204,34,female,2 years of college,29500
188787,54,male,grade 12,3500
76261,4,female,nursery school to grade 4,9999999
238671,32,female,grade 11,10000
195113,59,female,2 years of college,32500
114963,65,female,2 years of college,30900
184160,68,female,grade 12,2400


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [36]:
acs["inctot"] = acs["inctot"].replace(9999999, np.nan)

### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?


In [37]:
# Code I'd run, but which I'm commenting out so this notebook runs:

# acs["age"].average()

> I get a `TypeError`. Namely: `TypeError: 'Categorical' with dtype category does not support reduction 'mean'` It appears I'm getting the error because `age` is being stored as a Categorical rather than as a numeric type, and `mean` only works for numeric data.

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. What are those categories?

In [38]:
results = dict()

# One way to find problems:

# Make string so can use `str.isnumeric`
acs["age"] = acs["age"].astype("str")

# us `str.isnumeric`
problems = acs.loc[~acs["age"].str.isnumeric(), "age"].unique()
problems

array(['less than 1 year old', '90 (90+ in 1980 and 1990)'], dtype=object)

In [39]:
# Or just poke around!

# I could run this code, but I'm commenting it out so this notebook isn't awful to look at:

# for i in acs.age.value_counts().index:
#     print(i)

In [40]:
results["ex5_age_young"] = problems[0]
results["ex5_age_old"] = problems[1]
print(
    f"The problematic values are '{results['ex5_age_young']}' and '{results['ex5_age_old']}'"
)

The problematic values are 'less than 1 year old' and '90 (90+ in 1980 and 1990)'


### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted has occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondent's actual age. We're only working with more recent data, so anyone with a value of `90 (90+ in 1980 and 1990)` can safely be assumed to be 90 years old.

In [41]:
acs["age"] = acs["age"].str.replace("less than 1 year old", "0", regex=True)
acs["age"] = acs["age"].str.replace("90 \(90\+ in 1980 and 1990\)", "90", regex=True)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/var/folders/fs/h_8_rwsn5hvg9mhp0txgc_s9v6191b/T/ipykernel_96212/3726935054.py:2: SyntaxWarning: invalid escape sequence '\('
  acs["age"] = acs["age"].str.replace("90 \(90\+ in 1980 and 1990\)", "90", regex=True)


In [42]:
# Check that worked
acs.loc[~acs["age"].str.isnumeric(), "age"].unique()

array([], dtype=object)

In [43]:
assert acs["age"].str.isnumeric().all()

### Exercise 7

Now convert age from a Categorical to numeric. Calculate the average age in this data. Round your answer to 1 decimal place.

In [44]:
acs["age"] = acs["age"].astype("int")
results["ex7_avg_age"] = acs["age"].mean()
print(f"The average age is {results['ex7_avg_age']:.1f}")

The average age is 41.3


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! 

Calculate the new average age. Round your answer to 1 decimal place.

Use this sample of people 18 and over for all subsequent exercises.

In [45]:
acs = acs[acs["age"] >= 18].copy()
results["ex8_avg_age"] = acs["age"].mean()
print(f"The average age of those above 18 is {results['ex8_avg_age']:.1f}")

The average age of those above 18 is 49.8


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate **the number of respondents in the dataset** with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college (and if you're not from the US: college begins after completing Grade 12).

In [46]:
acs["educ"].value_counts(dropna=False)

educ
grade 12                     92576
4 years of college           47212
1 year of college            38746
5+ years of college          29801
2 years of college           20753
grade 5, 6, 7, or 8           5975
grade 11                      5816
grade 10                      4078
n/a or no schooling           3644
grade 9                       3145
nursery school to grade 4     1288
Name: count, dtype: int64

In [47]:
# Could do with two conditions and |
acs["college_degree"] = (acs["educ"] == "4 years of college") | (
    acs["educ"] == "5+ years of college"
)
acs["college_degree"].value_counts(dropna=False)

college_degree
False    176021
True      77013
Name: count, dtype: int64

In [48]:
# Or you could do with `.isin`
acs["college_degree"] = acs["educ"].isin(["4 years of college", "5+ years of college"])
acs["college_degree"].value_counts(dropna=False)

college_degree
False    176021
True      77013
Name: count, dtype: int64

In [49]:
# Check it worked
acs.loc[acs["college_degree"] == 1, "educ"].value_counts()

educ
4 years of college           47212
5+ years of college          29801
n/a or no schooling              0
nursery school to grade 4        0
grade 5, 6, 7, or 8              0
grade 9                          0
grade 10                         0
grade 11                         0
grade 12                         0
1 year of college                0
2 years of college               0
Name: count, dtype: int64

In [50]:
# Remember booleans are just special cases of 0 and 1, so
# the count that's true we can get with `sum`
results["ex9_num_college"] = acs["college_degree"].sum()
print(
    f"There are {results['ex9_num_college']:,.0f} people in our data with a college degree."
)

There are 77,013 people in our data with a college degree.


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [51]:
pd.crosstab(acs["college_degree"], acs["sex"])

sex,male,female
college_degree,,
False,85821,90200
True,36181,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

What share (between 0 and 1) of men have college degrees in this data? Round your answer to three decimal places.

What share of women have college degrees in this data? Round your answer to three decimal places.

In [52]:
tab = pd.crosstab(acs["college_degree"], acs["sex"], normalize="columns")
tab

sex,male,female
college_degree,,
False,0.703439,0.688381
True,0.296561,0.311619


In [53]:
tab_index = pd.crosstab(acs["college_degree"], acs["sex"], normalize="index")
tab_index

sex,male,female
college_degree,,
False,0.487561,0.512439
True,0.469804,0.530196


In [54]:
tab_all = pd.crosstab(acs["college_degree"], acs["sex"], normalize=True)
tab_all

sex,male,female
college_degree,,
False,0.339168,0.356474
True,0.142989,0.161370


In [55]:
for g in ["male", "female"]:
    results[f"ex11_share_{g}_w_degrees"] = tab.loc[True, g]
    print(
        f"The share of {g}s with college degrees is "
        f"{results[f'ex11_share_{g}_w_degrees']:.3f}"
    )

The share of males with college degrees is 0.297
The share of females with college degrees is 0.312


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. 

Among the older cohort (those 40 and over), is the share of men who have college degrees greater than the share of women who have college degrees?

Among the younger cohort (those under 40), is the share of men who have college degrees greater than the share of women who have college degrees?

What does this suggest about how educational attainment is changing over time in the United States?

In [56]:
older = acs[40 <= acs["age"]]
older_table = pd.crosstab(older["college_degree"], older["sex"], normalize="columns")
older_table

sex,male,female
college_degree,,
False,0.682123,0.699144
True,0.317877,0.300856


In [57]:
older_gap = older_table.loc[True, "male"] - tab.loc[True, "female"]
print(
    f"the gap between men and women's college attainment"
    f" for those OVER 40 is {older_gap:.3f}"
)

the gap between men and women's college attainment for those OVER 40 is 0.006


In [58]:
younger = acs[acs["age"] < 40]
younger_table = pd.crosstab(
    younger["college_degree"], younger["sex"], normalize="columns"
)
tab

sex,male,female
college_degree,,
False,0.703439,0.688381
True,0.296561,0.311619


In [59]:
younger_gap = tab.loc[True, "male"] - tab.loc[True, "female"]
print(
    f"the gap between men and women's college attainment"
    f" for those UNDER 40 is {younger_gap:.3f}"
)

the gap between men and women's college attainment for those UNDER 40 is -0.015


In [60]:
print(
    f"The male-female gap has gone from {older_gap:.3f} to {younger_gap:.3f}. "
    f"Thus the absolute change from older to younger cohorts is {np.abs(younger_gap) - np.abs(older_gap):.3f}."
)

The male-female gap has gone from 0.006 to -0.015. Thus the absolute change from older to younger cohorts is 0.009.


In [61]:
results["ex12_comparing"] = "the absolute difference has increased"